### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#school_data_complete.head()
#finding unique number of schools
#schools is an array with each unique school name
schools = len(school_data_complete["school_name"].unique())

#finding number of students
#need to go on student ID since names repeat.  Using unique to make sure don't have duplicates
students = len(school_data_complete["Student ID"].unique())

school_data_complete['pass read']=school_data_complete['reading_score'].ge(70) #ge in pandas is greater than equal to
school_data_complete['pass math']=school_data_complete['math_score'].ge(70)
school_data_complete['pass both']=school_data_complete.loc[:,"pass read"]&school_data_complete.loc[:,"pass math"]

#finding total budget
#first create a group on school name
#then from group find unique number of budgets as an array
#convert array into dataframe for easier manipulation
grouped_schools = school_data_complete.groupby(["school_name"])
school_budgets = grouped_schools["budget"].unique()
school_budgets_df = pd.DataFrame({"school_budget": school_budgets})
school_budgets_df = school_budgets_df.copy()#creating copy to save memory (per 06-Ins_group )
school_budgets_df["school_budget"] = school_budgets_df.loc[:,"school_budget"].astype(float) #convert to float
total_budget = school_budgets_df["school_budget"].sum() #sum column of school budget to get total budget

ave_read_score = round((school_data_complete["reading_score"].mean()),2)
ave_math_score = round((school_data_complete["math_score"].mean()),2)

#number passing math
num_math_pass = 0
#Calculate the percentage of students with a passing math score (70 or greater)
math_pass = school_data_complete["math_score"].ge(70)
for row in math_pass:
    if row == True:
        num_math_pass += 1

per_pass_math = round((num_math_pass/students*100),2)        

#number passing reading
num_read_pass = 0
#Calculate the percentage of students with a passing math score (70 or greater)
read_pass = school_data_complete["reading_score"].ge(70)
for row in read_pass:
    if row == True:
        num_read_pass += 1
        
per_pass_read = round((num_read_pass/students*100),2)
#print(f"Percent of students greater than 70 in reading {per_pass_read}")

num_read_and_math_pass = 0

for i in range (0,len(read_pass)):
    if read_pass[i] and math_pass[i]:
        num_read_and_math_pass += 1

per_pass_read_math = round((num_read_and_math_pass/students*100),2)

#print(f"Percent of students greater than 70 in reading and math {per_pass_read_math}")

summary_1_df = pd.DataFrame({"Num Schools":[schools],
                            "Students":[students],
                            "Total Budget": "$"+str(int(total_budget)),
                            "Ave_Math": ave_math_score,
                            "Ave_Read": ave_read_score,
                            "%Pass_Math": str(per_pass_math)+"%",
                            "%Pass_Read": str(per_pass_read)+"%",
                            "%Pass_Both":str(per_pass_read_math)+"%"})

print("Overall District Summary")
print("")
print(summary_1_df.to_string(col_space = 12, index = False,
                            index_names=False, justify = "center",max_colwidth = 20))

Overall District Summary

 Num Schools   Students   Total Budget   Ave_Math     Ave_Read    %Pass_Math   %Pass_Read   %Pass_Both 
     15          39170      $24649428      78.99        81.88       74.98%       85.81%       65.17%   


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [18]:
#getting school budgets
school_budget = grouped_schools["budget"].unique()
school_budget = school_budget.astype(float)

school_students = grouped_schools["Student ID"].count()

school_pass_math = grouped_schools["pass math"].sum()
school_pass_read = grouped_schools["pass read"].sum()
school_pass_both = grouped_schools["pass both"].sum()

school_type = grouped_schools["type"].unique()

school_mean_math = grouped_schools["math_score"].mean()
school_mean_read = grouped_schools["reading_score"].mean()

#creating dataframe
school_detail = pd.DataFrame(school_budget)
school_detail['students'] = school_students
school_detail['type'] = school_type
school_detail['budg/stud']=school_detail['budget']/school_detail['students']
school_detail['ave math']=round(school_mean_math,2)
school_detail['ave read']=round(school_mean_read,2)
school_detail['% pass math'] = round((school_pass_math/school_students*100),2)
school_detail['% pass read'] = round((school_pass_read/school_students*100),2)
school_detail["% pass both"] = round((school_pass_both/school_students*100),2)

#copy dataframe for formatting
school_detail1 = school_detail.copy()
school_detail1["budget"]=school_detail1["budget"].map('{:.0f}'.format)
school_detail1["budg/stud"]=school_detail1["budg/stud"].map('${:.0f}'.format)
school_detail1["% pass math"]=school_detail1["% pass math"].map('{:.1f}%'.format)
school_detail1["% pass read"]=school_detail1["% pass read"].map('{:.1f}%'.format)
school_detail1["% pass both"]=school_detail1["% pass both"].map('{:.1f}%'.format)
print("")
print("School Summaries")
print("")
print(school_detail1.to_string(columns = ["type", "students","budget", "budg/stud", "ave math", "ave read", 
                                        "% pass math", "% pass read", "% pass both"],col_space = 10, 
                            index_names=False, justify = "center",max_colwidth = 20))
del school_detail1


School Summaries

                         type     students    budget   budg/stud   ave math   ave read  % pass math % pass read % pass both
Bailey High School    [District]    4976      3124928     $628       77.05      81.03       66.7%       81.9%       54.6%  
Cabrera High School    [Charter]    1858      1081356     $582       83.06      83.98       94.1%       97.0%       91.3%  
Figueroa High School  [District]    2949      1884411     $639       76.71      81.16       66.0%       80.7%       53.2%  
Ford High School      [District]    2739      1763916     $644       77.10      80.75       68.3%       79.3%       54.3%  
Griffin High School    [Charter]    1468       917500     $625       83.35      83.82       93.4%       97.1%       90.6%  
Hernandez High Sc...  [District]    4635      3022020     $652       77.29      80.93       66.8%       80.9%       53.5%  
Holden High School     [Charter]     427       248087     $581       83.80      83.81       92.5%       96.2%    

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [28]:
Top_school = school_detail.sort_values(by="% pass both", ascending = False)

Top_five = Top_school.head(5)

print("Top Five Schools by % Passing both Math and Reading")

print("")

Top_five1 = Top_five.copy()
Top_five1["budget"]=Top_five["budget"].map('${:,.0f}'.format)
Top_five1["budg/stud"]=Top_five["budg/stud"].map('${:.0f}'.format)
Top_five1["% pass math"]=Top_five["% pass math"].map('{:.1f}%'.format)
Top_five1["% pass read"]=Top_five["% pass read"].map('{:.1f}%'.format)
Top_five1["% pass both"]=Top_five["% pass both"].map('{:.1f}%'.format)

Top_five1 = Top_five1.rename(columns={"% pass math":"%pass_math", "% pass read":"%pass_read", "% pass both":"%pass_both"})

print(Top_five1.to_string(columns = ["type","students","budget","budg/stud",
                                       "ave math","ave read","%pass_math","%pass_read","%pass_both"], col_space = 10, 
                            index_names=False, justify = "center"))

Top Five Schools by % Passing both Math and Reading

                       type     students     budget   budg/stud   ave math   ave read  %pass_math %pass_read %pass_both
Cabrera High School  [Charter]    1858     $1,081,356    $582       83.06      83.98      94.1%      97.0%      91.3%  
Thomas High School   [Charter]    1635     $1,043,130    $638       83.42      83.85      93.3%      97.3%      91.0%  
Griffin High School  [Charter]    1468       $917,500    $625       83.35      83.82      93.4%      97.1%      90.6%  
Wilson High School   [Charter]    2283     $1,319,574    $578       83.27      83.99      93.9%      96.5%      90.6%  
Pena High School     [Charter]     962       $585,858    $609       83.84      84.04      94.6%      96.0%      90.5%  


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [31]:
Bottom_five = Top_school.tail(5)
Bottom_five = Bottom_five.sort_values(by="% pass both", ascending = True)
print("Bottom Five Schools by % Passing both Reading and Math")
print("")

#Score_by_Spend_df['ave math'] = Score_by_Spend_df['ave math'].map('{:.2f}'.format)

Bottom_five1 = Bottom_five.copy()
Bottom_five1["budget"]=Bottom_five["budget"].map('${:.0f}'.format)
Bottom_five1["budg/stud"]=Bottom_five["budg/stud"].map('${:.0f}'.format)
Bottom_five1["% pass math"]=Bottom_five["% pass math"].map('{:.1f}%'.format)
Bottom_five1["% pass read"]=Bottom_five["% pass read"].map('{:.1f}%'.format)
Bottom_five1["% pass both"]=Bottom_five["% pass both"].map('{:.1f}%'.format)

Bottom_five1 = Bottom_five1.rename(columns={"% pass math":"%pass_math", "% pass read":"%pass_read", "% pass both":"%pass_both"})

print(Bottom_five1.to_string(columns = ["type","students","budget","budg/stud",
                                       "ave math","ave read","%pass_math","%pass_read","%pass_both"], col_space = 10, 
                            index_names=False, justify = "center"))

Bottom Five Schools by % Passing both Reading and Math

                          type     students    budget   budg/stud   ave math   ave read  %pass_math %pass_read %pass_both
Rodriguez High School  [District]    3999     $2547363     $637       76.84      80.74      66.4%      80.2%      53.0%  
Figueroa High School   [District]    2949     $1884411     $639       76.71      81.16      66.0%      80.7%      53.2%  
Huang High School      [District]    2917     $1910635     $655       76.63      81.18      65.7%      81.3%      53.5%  
Hernandez High School  [District]    4635     $3022020     $652       77.29      80.93      66.8%      80.9%      53.5%  
Johnson High School    [District]    4761     $3094650     $650       77.07      80.97      66.1%      81.2%      53.5%  


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [32]:
Math_9th_df = school_data_complete[school_data_complete["grade"] == "9th"]
Math_9th_df = Math_9th_df[["school_name","math_score"]]
Math_9th_group = Math_9th_df.groupby("school_name")
Math_9th=round(Math_9th_group.mean(),2)
Math_by_Grade = Math_9th.rename(columns = {"math_score":"math_9th"})

Math_10th_df = school_data_complete[school_data_complete["grade"] == "10th"]
Math_10th_df = Math_10th_df[["school_name","math_score"]]
Math_10th_group = Math_10th_df.groupby("school_name")
Math_10th=round(Math_10th_group.mean(),2)
#Math_10th = Math_10th.rename(columns = {"math_score":"math_10th"})

Math_11th_df = school_data_complete[school_data_complete["grade"] == "11th"]
Math_11th_df = Math_11th_df[["school_name","math_score"]]
Math_11th_group = Math_11th_df.groupby("school_name")
Math_11th=round(Math_11th_group.mean(),2)

Math_12th_df = school_data_complete[school_data_complete["grade"] == "12th"]
Math_12th_df = Math_12th_df[["school_name","math_score"]]
Math_12th_group = Math_12th_df.groupby("school_name")
Math_12th=round(Math_12th_group.mean(),2)

Math_by_Grade["math_10th"] = Math_10th
Math_by_Grade["math_11th"] = Math_11th
Math_by_Grade["math_12th"] = Math_12th
print(Math_by_Grade.to_string(col_space = 12, justify = "center",index = True, index_names = False))

                        math_9th     math_10th    math_11th    math_12th 
Bailey High School        77.08        77.00        77.52        76.49   
Cabrera High School       83.09        83.15        82.77        83.28   
Figueroa High School      76.40        76.54        76.88        77.15   
Ford High School          77.36        77.67        76.92        76.18   
Griffin High School       82.04        84.23        83.84        83.36   
Hernandez High School     77.44        77.34        77.14        77.19   
Holden High School        83.79        83.43        85.00        82.86   
Huang High School         77.03        75.91        76.45        77.23   
Johnson High School       77.19        76.69        77.49        76.86   
Pena High School          83.63        83.37        84.33        84.12   
Rodriguez High School     76.86        76.61        76.40        77.69   
Shelton High School       83.42        82.92        83.38        83.78   
Thomas High School        83.59       

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [33]:
Read_9th_df = school_data_complete[school_data_complete["grade"] == "9th"]
Read_9th_df = Read_9th_df[["school_name","reading_score"]]
Read_9th_group = Read_9th_df.groupby("school_name")
Read_9th=round(Read_9th_group.mean(),2)
Read_by_Grade = Read_9th.rename(columns = {"reading_score":"reading_9th"})

Read_10th_df = school_data_complete[school_data_complete["grade"] == "10th"]
Read_10th_df = Read_10th_df[["school_name","reading_score"]]
Read_10th_group = Read_10th_df.groupby("school_name")
Read_10th=round(Read_10th_group.mean(),2)

Read_11th_df = school_data_complete[school_data_complete["grade"] == "11th"]
Read_11th_df = Read_11th_df[["school_name","reading_score"]]
Read_11th_group = Read_11th_df.groupby("school_name")
Read_11th=round(Read_11th_group.mean(),2)

Read_12th_df = school_data_complete[school_data_complete["grade"] == "12th"]
Read_12th_df = Read_12th_df[["school_name","reading_score"]]
Read_12th_group = Read_12th_df.groupby("school_name")
Read_12th=round(Read_12th_group.mean(),2)

Read_by_Grade["reading_10th"] = Read_10th
Read_by_Grade["reading_11th"] = Read_11th
Read_by_Grade["reading_12th"] = Read_12th
print(Read_by_Grade.to_string(col_space = 12, justify = "center",index = True, index_names = False))

                       reading_9th  reading_10th  reading_11th  reading_12th
Bailey High School        81.30         80.91         80.95         80.91   
Cabrera High School       83.68         84.25         83.79         84.29   
Figueroa High School      81.20         81.41         80.64         81.38   
Ford High School          80.63         81.26         80.40         80.66   
Griffin High School       83.37         83.71         84.29         84.01   
Hernandez High School     80.87         80.66         81.40         80.86   
Holden High School        83.68         83.32         83.82         84.70   
Huang High School         81.29         81.51         81.42         80.31   
Johnson High School       81.26         80.77         80.62         81.23   
Pena High School          83.81         83.61         84.34         84.59   
Rodriguez High School     80.99         80.63         80.86         80.38   
Shelton High School       84.12         83.44         84.37         82.78   

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [34]:

bins = [0,584,629,644,675] #setting binning ranges

labels = [1,2,3,4]
school_detail['binned'] = pd.cut(school_detail['budg/stud'], bins,labels=labels, include_lowest = False)

bin_1_df = ( school_detail[school_detail["binned"] == 1]) # collect all schools bin 1
bin_2_df = ( school_detail[school_detail["binned"] == 2]) # collect all schools bin 2
bin_3_df = ( school_detail[school_detail["binned"] == 3]) # collect all schools bin 3
bin_4_df = ( school_detail[school_detail["binned"] == 4]) # collect all schools bin 4

bin_1 = bin_1_df.mean(axis = 0) #average of all columns
bin_2 = bin_2_df.mean(axis=0)
bin_3 = bin_3_df.mean(axis=0)
bin_4 = bin_4_df.mean(axis=0)



bins_label = [str(f" ${bins[0]} to ${bins[1]}"), str(f" ${bins[1]} to ${bins[2]}"),
              str(f" ${bins[2]} to ${bins[3]}"), str(f" ${bins[3]} to ${bins[4]}")]

Score_by_Spend_df = pd.DataFrame([bin_1[3:8], bin_2[3:8], bin_3[3:8], bin_4[3:8]])

#df['cost'] = df['cost'].map('${:,.2f}'.format)


Score_by_Spend_df.insert(0,"budg/std",bins_label)

Score_by_Spend_df['ave math'] = Score_by_Spend_df['ave math'].map('{:.2f}'.format)
Score_by_Spend_df['ave read'] = Score_by_Spend_df['ave read'].map('{:.2f}'.format)
Score_by_Spend_df['% pass math'] = Score_by_Spend_df['% pass math'].map('{:.2f}%'.format)
Score_by_Spend_df['% pass read'] = Score_by_Spend_df['% pass read'].map('{:.2f}%'.format)
Score_by_Spend_df['% pass both'] = Score_by_Spend_df['% pass both'].map('{:.2f}%'.format)

print(Score_by_Spend_df.to_string(col_space = 12, justify = "center",index = False))






   budg/std      ave math     ave read   % pass math  % pass read  % pass both 
    $0 to $584     83.45        83.94       93.46%       96.61%       90.37%   
  $584 to $629     81.90        83.16       87.13%       92.72%       81.42%   
  $629 to $644     78.52        81.62       73.48%       84.39%       62.86%   
  $644 to $675     77.00        81.03       66.16%       81.13%       53.53%   


## Scores by School Size

* Perform the same operations as above, based on school size.

In [35]:
#del school_detail["binned"]

max_student = school_detail["students"].max()
min_student = school_detail["students"].min()

bin_size = (max_student-min_student)/3

#bins = [int(round((min_student+bin_size*i),0)) for i in range(0,4)]
bins = (0,1000,2000,5000)

labels = [1,2,3]
school_detail['binned'] = pd.cut(school_detail['students'], bins,labels=labels, include_lowest = True)

bin_1_df = ( school_detail[school_detail["binned"] == 1])
bin_2_df = ( school_detail[school_detail["binned"] == 2])
bin_3_df = ( school_detail[school_detail["binned"] == 3])

bin_1 = bin_1_df.mean(axis = 0) #average of columns
bin_2 = bin_2_df.mean(axis=0)
bin_3 = bin_3_df.mean(axis=0)


bins_label = [str(f" {bins[0]} to {bins[1]}"), str(f" {bins[1]} to {bins[2]}"),
              str(f" {bins[2]} to {bins[3]}")]

Score_by_Size_df = pd.DataFrame([bin_1[3:8], bin_2[3:8], bin_3[3:8]])

Score_by_Size_df.insert(0,"School Size",bins_label)

Score_by_Size_df['ave math'] = Score_by_Size_df['ave math'].map('{:.2f}'.format)
Score_by_Size_df['ave read'] = Score_by_Size_df['ave read'].map('{:.2f}'.format)
Score_by_Size_df['% pass math'] = Score_by_Size_df['% pass math'].map('{:.2f}%'.format)
Score_by_Size_df['% pass read'] = Score_by_Size_df['% pass read'].map('{:.2f}%'.format)
Score_by_Size_df['% pass both'] = Score_by_Size_df['% pass both'].map('{:.2f}%'.format)

print(Score_by_Size_df.to_string(col_space = 12, justify = "center",index = False))



 School Size     ave math     ave read   % pass math  % pass read  % pass both 
     0 to 1000     83.82        83.93       93.55%       96.10%       89.89%   
  1000 to 2000     83.37        83.87       93.60%       96.79%       90.62%   
  2000 to 5000     77.74        81.34       69.96%       82.77%       58.29%   


## Scores by School Type

* Perform the same operations as above, based on school type

In [36]:
bin_1_df = ( school_detail[school_detail["type"] == "Charter"])
bin_2_df = ( school_detail[school_detail["type"] == "District"])

bin_1 = bin_1_df.mean(axis = 0) #average of columns
bin_2 = bin_2_df.mean(axis=0)

bins_label = ["Charter", "District"]

Score_by_Type_df = pd.DataFrame([bin_1[3:8], bin_2[3:8]])

Score_by_Type_df.insert(0,"School Type",bins_label)

Score_by_Type_df['ave math'] = Score_by_Type_df['ave math'].map('{:.2f}'.format)
Score_by_Type_df['ave read'] = Score_by_Type_df['ave read'].map('{:.2f}'.format)
Score_by_Type_df['% pass math'] = Score_by_Type_df['% pass math'].map('{:.2f}%'.format)
Score_by_Type_df['% pass read'] = Score_by_Type_df['% pass read'].map('{:.2f}%'.format)
Score_by_Type_df['% pass both'] = Score_by_Type_df['% pass both'].map('{:.2f}%'.format)

print(Score_by_Type_df.to_string(col_space = 12, justify = "center",index = False))

School Type    ave math     ave read   % pass math  % pass read  % pass both 
   Charter       83.47        83.90       93.62%       96.59%       90.43%   
  District       76.96        80.97       66.55%       80.80%       53.67%   
